In [75]:
import os
import json
import subprocess
import datetime

In [103]:
# File path
working_dir = os.path.dirname(os.path.abspath(''))
v_json_export_path = working_dir + "\\src\\data\\processing-batch\\topic_model.json"
batch_dir = working_dir + "\\src\\data\\processing-batch"
bat_file_path = batch_dir + "\\Tokenizer.bat"
jvn_path = batch_dir + "\\JVnTextPro-3.0.3-executable.jar"
stop_word_file = batch_dir + "\\vietnamese-stopwords-dash.txt"
pmi_path = batch_dir + "\\pmi.json"

In [77]:
# Load topic model 
with open(v_json_export_path, "r", encoding="utf-8") as file:
    topic_model = json.load(file)

In [111]:
# Load pmi cho các từ
with open(pmi_path, "r", encoding="utf-8") as file:
    PMI = json.load(file)

In [78]:
# Lấy ra các hư từ
stop_word_file = open(stop_word_file, "r", encoding="utf-8")
stop_words = stop_word_file.read()
stop_words = stop_words.split("\n")
stop_words.append("ảnh")

In [79]:
def get_noun_word(words: list, noun_type: str, delimiter_text: str, stop_words: list) -> list:
    '''
    Lấy ra các danh từ trong tập các từ
    
    Param:
    --------------------------------------------
    words: tất cả các từ trong văn bản, còn chứa pos
    noun_type: loại danh từ cần tách
    delimiter_text: ký hiệu phân loại của từ
    stop_words: danh sách các hư từ
    
    Return:
    ----------------------------------------------
    Trả về list các từ có loại noun_type đã được chuyển về chữ thường
    '''
    
    noun_words = []
    for text_split in words:
        text_split = text_split
        text_and_type = text_split.split(delimiter_text)
        type = text_and_type.pop()
        if type == noun_type:
            text = text_and_type.pop()
            # Nếu từ không phải hư từ thì đưa vào tập noun_word
            if text.lower() not in stop_words:
                noun_words.append(text.lower())
    return noun_words

In [80]:
def pos_tagging_document(document:str, file_input_name: str | None = None):
    ''' 
    Đánh dấu từ loại cho văn bảng
    
    Param
    ------------------------------------------
    document: văn bảng dạng text hoặc file
    file_input_name: nếu là None thì văn bảng là dạng text và ngược lại
    '''
    
     # Chuyển text thành file để xử lý bằng jvnpro
    if file_input_name == None:
        file_input_name = f"Temp{datetime.datetime.now().strftime('%Y%m%d%H%M%S')}"
        input_file_name = batch_dir + f"\\data-process\\{file_input_name}.txt"
        document = document.replace("\n", " ")
    
        # Chuyển văn bảng ra thành file
        with open(input_file_name, "x", encoding="utf-8") as file:
            file.write(document)
    # TODO: xử lý văn bảng dạng file
        
    subprocess.call([bat_file_path, jvn_path, input_file_name])
    
    # File output của tool
    tagging_document = ""
    input_file_name_pro = batch_dir + f"\\data-process\\{file_input_name}.txt.pro"
    with open(input_file_name_pro, "r", encoding="utf-8") as output_file:
        tagging_document = output_file.read()
    
    # Xoá các file được tạo ra bởi tool
    os.remove(input_file_name)
    os.remove(input_file_name_pro)
    
    return tagging_document.replace("\n", " ")

In [81]:
def count_all_terminology(tagging_document: str, topic_model: dict, topic: str | None) -> dict:
    '''
    Đếm số lần xuất hiện của các thuật ngữ và trả về dict
    
    Param
    -----------------------------------------
    tagging_document: văn bản đã được đánh dấu từ loại
    topic: chủ đề của văn bản (nếu không biết trước thì đặt là None)
    
    Return
    -----------------------------------------
    dict: chứa số lần xuất hiện của tất cả thuật ngữ 
    {terminology: count, all: count}
    '''
    
    terminologys = {}
    
    words =  tagging_document.split(" ")
    noun_words = get_noun_word(words, "N", "/", stop_words)
    
    # TODO: xử lý khi không có topic cụ thể
    if topic == None:
        pass
    else:
        # Đếm các thuật ngữ có trong văn bản
        model_terminologys = [word[0] for word in topic_model[f'{topic}']]
        count_all = 0
        for noun in noun_words: 
            if noun in model_terminologys:
                count_all += 1
                if noun in terminologys.keys():
                    terminologys[f'{noun}'] += 1
                else:
                    terminologys[f'{noun}'] = 1
    
    terminologys['terminologies_count_all'] = count_all
    return terminologys

In [82]:
def weight_terminology(sentences: str, terminology_count: dict) -> float:
    '''
    Tính trọng số của các thuật ngữ trong một câu
    
    Param
    ---------------------------------------------
    sentences: câu cần tính trọng số
    terminology_count: tập các thuật ngữ
    
    Return
    --------------------------------------------
    Trọng số: float
    '''
    words = sentences.split(" ")
    noun_word = get_noun_word(words, "N", "/", stop_words)
    
    # Trọng số của câu
    weight = 0
    
    # Lấy ra các thuật ngữ trong câu
    noun_word = [noun for noun in noun_word if noun in terminology_count.keys()]
    
    # Không có thuật ngữ nào trong câu
    if len(noun_word) == 0:
        return 0
    
    # Tính trọng số cho từng thuật ngữ
    noun_word.sort()
    noun_weight = 1
    noun_current = noun_word[0]
    for i in range(1, len(noun_word)):
        if noun_word[i] != noun_current:
            # Số lần xuất hiện của thuật ngữ chia cho toàn bộ số lần xuất hiện của các thuật ngữ khác
            weight += noun_weight / terminology_count['terminologies_count_all']
            noun_current = noun_word[i]
            noun_weight = 1
        else:
            noun_weight += 1
    weight += noun_weight / terminology_count['terminologies_count_all']
    return weight

In [83]:
def cal_sentences_weight(tagging_document: str, terminology_count:dict) -> list:
    '''
    Tính trọng số của các câu trong văn bản
    
    Params
    ----------------------------
    terminology_word: thuật ngữ trong câu
    terminology_count: các thuật ngữ có trong văn bản
    
    Return
    -----------------------------
    Trả về list có định dạng như sau:
    [
        {
            id: id của câu,
            content: nội dung câu,
            weight: trọng số của câu dựa vào thuật ngữ
        }
    ]
    '''
    
    sentences = tagging_document.split("./.")
    weighted_sentences = []
    for i in range(len(sentences)):
        weight = weight_terminology(sentences[i], terminology_count)
        
        weighted_sentences.append({
            'id' : f'S{i+1}',
            'content': sentences[i],
            'weight': weight,
        })
    
    return weighted_sentences
    

In [84]:
document = '''Ngày 15/8, Ban Quản lý rừng phòng hộ huyện Nghi Lộc cho biết dịch sâu róm xuất hiện tại cánh rừng thông trên địa bàn từ giữa tháng 7. Vòng đời sâu róm khoảng 50 ngày, hiện đã đến thế hệ thứ ba.

Đến nay, 300 ha rừng thông ở các xã Nghi Yên, Nghi Tiến, Nghi Quang, Nghi Xá bị sâu ăn trụi lá, mật độ khoảng 350-400 con một cây. 450 ha rừng thông rải rác trải dài trên 17 xã khác trong vùng cũng bị sâu tấn công với mức độ trung bình, mật độ 150-200 con một cây. Một số điểm nhỏ lẻ là 10-30 con một cây.
Ông Trần Văn Trường, Phó ban Quản lý rừng phòng hộ huyện Nghi Lộc, cho biết vừa qua trời nhiều sương mù, nắng mưa thất thường, độ ẩm cao, tạo điều kiện cho sâu róm sinh trưởng mạnh. Hiện sâu ăn, cắn phá lá thông một chu kỳ, nếu để sâu phát triển thêm nhiều chu kỳ sẽ có thể gây chết cây.

"Chúng tôi đã cử cán bộ phun thuốc trên diện tích 142 ha. Thời điểm này sâu bắt đầu đóng kén, chuẩn bị vòng đời mới nên tạm dừng phun. Đơn vị đang chuẩn bị đèn và các vật dụng làm điểm đèn để bẫy, bắt sâu trưởng thành nở ra bướm", ông Trường nói.
Ban Quản lý rừng phòng hộ huyện Nghi Lộc được giao nhiệm vụ quản lý, bảo vệ và phát triển hơn 5.000 ha đất lâm nghiệp gồm đất rừng quy hoạch phòng hộ, rừng sản xuất, trải dài trên 17 xã.

Sâu róm hay còn gọi là sâu lông, khi trưởng thành có lông chứa độc tố.'''

In [85]:
# Tagging
tagging_document = pos_tagging_document(document, None)
# Đếm các thuật ngữ
terminologies = count_all_terminology(tagging_document, topic_model, 'ThoiSu')

In [86]:
# Tính trọng số của câu
sentences_weights = cal_sentences_weight(tagging_document, terminologies)
sentences_weights

[{'id': 'S1',
  'content': 'Ngày/N 15/8/M ,/, Ban_Quản_lý/N rừng_phòng_hộ/N huyện/N Nghi_Lộc/Np cho_biết/Np dịch/N sâu_róm/N xuất_hiện/V tại/E cánh/N rừng/N thông/A trên/A địa_bàn/N từ/E giữa/M tháng/N 7/M ',
  'weight': 0.125},
 {'id': 'S2',
  'content': ' Vòng/N đời/N sâu_róm/N khoảng/N 50/M ngày/N ,/, hiện/Np đã/R đến/V thế_hệ/N thứ_ba/N ',
  'weight': 0.03125},
 {'id': 'S3',
  'content': ' Đến/E nay/P ,/, 300/M ha/Nu rừng/N thông/A ở/E các/L xã/N Nghi/Np Yên/Np ,/, Nghi_Tiến/Np ,/, Nghi/Np Quang/Np ,/, Nghi/Np Xá/Np bị/V sâu/A ăn/V trụi/A lá/A ,/, mật_độ/N khoảng/N 350-400/M con/Nc một/M cây/X ',
  'weight': 0.09375},
 {'id': 'S4',
  'content': ' 450/M ha/Nu rừng/N thông/V rải_rác/A trải/V dài/A trên/A 17/M xã/N khác/A trong_vùng/A cũng/R bị/V sâu/A tấn_công/V với/E mức_độ/N trung_bình/A ,/, mật_độ/N 150-200/M con/Np một/M cây/X ',
  'weight': 0.125},
 {'id': 'S5',
  'content': ' Một_số_điểm/Nc nhỏ/N lẻ/Np là/C 10-30/M con/Np một/M cây/X ',
  'weight': 0},
 {'id': 'S6',
  'content'

In [123]:
# Lấy ra các câu trong văn bản
sentences = tagging_document.split("./.")
# Tính độ tương đồng của câu
sentences_similarity = {}
max = 0
min = 0
for i in range(len(sentences) - 1):
    for j in range(1, len(sentences)):
        similarity = 0
        words_sen1 = get_noun_word(sentences[i].split(" "),"N","/",stop_words)
        words_sen2 = get_noun_word(sentences[j].split(" "),"N","/",stop_words)
        words_sen1.sort()
        words_sen2.sort()
        for word1 in words_sen1:
            for word2 in words_sen2:
                word_pair = word1 + " " + word2
                if word_pair in PMI.keys():
                    similarity += PMI[f'{word_pair}']
                    
        if max < similarity:
            max = similarity
        
        if min > similarity:
            min = similarity
        sentences_similarity[f'S{i+1} S{j+1}'] = similarity

# Chuẩn hoá lại độ tương đồng
for key, value in sentences_similarity.items():
    sentences_similarity[f'{key}'] = (value - min) / (max - min)

In [152]:
def concat_list_summary_sentences(summary_sentences: list) -> str:
    '''
    Cộng chuỗi các câu để tóm tắt
    
    Param
    -------------------------------
    summary_sentences: danh sách các câu
    
    Return
    -------------------------------
    str: Văn bảng tóm tắt theo thứ tự
    '''
    
    summary_document = ''
    summary_sentences = sorted(summary_sentences, key=lambda x:x[0])
    
    for summary_sentence in summary_sentences:
        summary_document += summary_sentence[1] + '. '
    
    return summary_document

In [159]:
# Xây dựng mô hình đồ thị
# Tỉ lệ tóm tắt
r_threshold = 0.4

# Tỉ lệ tương đồng
alpha = 0.2

sentences = tagging_document.split("./.")
sorted_sentences_weights = sorted(sentences_weights, key=lambda x:x['weight'], reverse=True)
choice_sentences = []
final_summary = []
for sentence_weight in sorted_sentences_weights:
    if len(concat_list_summary_sentences(final_summary))/len(document) > r_threshold:
        break 
    sentence_index = int(sentence_weight['id'][1:]) - 1
    sentence = sentences[sentence_index]
    
    # Xác định xem có dùng câu này không
    ok = False
    if len(choice_sentences) == 0:
        ok = True
    else:
        # Kiểm tra độ tương đồng của các câu
        for s_choice_index in choice_sentences:
            if (s_choice_index > sentence_index):
                id = f'S{sentence_index + 1} S{s_choice_index + 1}'
            else:
                id = f'S{s_choice_index + 1} S{sentence_index + 1}'
        
        if sentences_similarity[f'{id}'] < alpha:
            ok = True
    
    if ok:
        choice_sentences.append(sentence_index)
        # Xoá các loại từ và dấu _
        original_sentence = " "
        for word in sentence.split(" "):
            original_sentence += word.split("/")[0].replace('_', ' ') + ' '
        final_summary.append([sentence_index, original_sentence.strip()])

In [160]:
final_summary

[[9,
  'Ban Quản lý rừng phòng hộ huyện Nghi Lộc được giao nhiệm vụ quản lý , bảo vệ và phát triển hơn 5.000 ha đất lâm nghiệp gồm đất rừng quy hoạch phòng hộ , rừng sản xuất , trải dài trên 17 xã'],
 [8,
  'Đơn vị đang chuẩn bị đèn và các vật dụng làm điểm đèn để bẫy , bắt sâu trưởng thành nở ra bướm " , ông Trường nói'],
 [2,
  'Đến nay , 300 ha rừng thông ở các xã Nghi Yên , Nghi Tiến , Nghi Quang , Nghi Xá bị sâu ăn trụi lá , mật độ khoảng 350-400 con một cây'],
 [1, 'Vòng đời sâu róm khoảng 50 ngày , hiện đã đến thế hệ thứ ba'],
 [4, 'Một số điểm nhỏ lẻ là 10-30 con một cây']]

In [ ]:
concat_list_summary_sentences(final_summary)

In [3]:
from bert_score import BERTScorer
from transformers.models.bert.modeling_bert import BertModel,BertForMaskedLM
scorer = BERTScorer(lang='vi')
_, _, F1_1 = scorer.score(["Nhan"], ["N"])
F1_1

c:\Person\CaoHoc\Project_ForAll\venv\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


tensor([0.7095])